In [1]:
!bjobs -q all_corradin -J dask-worker| wc -l

Job <dask-worker> is not found in queue <all_corradin>
0


In [2]:
# kill any remnants of dask worker
!bkill -q all_corradin -J dask-worker 0
!rm -rf worker-*
!rm -rf *.lock

Job <dask-worker> is not found in queue <all_corradin>


In [3]:
import dask
import dask_jobqueue
print(dask.__version__)
print(dask_jobqueue.__version__)

2021.09.1
0.7.3


In [4]:
#start new cluster

from dask_jobqueue import LSFCluster
from dask.distributed import Client, progress

#three different configurations I have tried

cores_per_machine = 47
processes_per_worker = 4
workers_per_machine = round(cores_per_machine/processes_per_worker)

cluster = LSFCluster(queue='all_corradin',
                         cores= cores_per_machine,
                         processes = workers_per_machine,
                         walltime='9000:00',
                         memory='240GB',
                         local_directory = "/tmp",
                         job_extra=['-o /dev/null', '-e /dev/null'],
                         scheduler_options={"port" : 8789, "dashboard_address": 8790},
                         )

scale_num = cores_per_machine * workers_per_machine

cluster.scale(scale_num) # this should be the total number of workers

#----
## config 1, 10 workers, 47 threads each, should see 10 LSF jobs:
# cores_per_machine = 47
# workers_per_machine = 10

# cluster = LSFCluster(queue='all_corradin',
#                          cores= cores_per_machine,
#                          processes = workers_per_machine,
#                          walltime='9000:00',
#                          memory='220GB',
#                          local_directory = "/tmp",
#                          job_extra=['-o /dev/null', '-e /dev/null'],
#                          scheduler_port = 8789,
#                          worker_dashboard_address = 8790
#                          )

### this should be the total number of workers, in this case, since it's 
# scale_num = 47 * 11
# cluster.scale(scale_num) 

###############--------------
#cluster.scale(100)
## config 2, 10 workers, each worker has 10 processes, so 400/10 = 4 threads per process:
## should see 10 LSF jobs

# cluster = LSFCluster(queue='all_corradin',
#                          cores= 47,
#                          processes = 10 , 
#                          walltime='5000:00',
#                          memory='200GB',
#                          #local_directory = "/mnt/ramdisk",
#                          job_extra=['-o /dev/null', '-e /dev/null']
#                          )
# cluster.scale(400)

## config 3,  100 workers, each worker has 1 process, 4 threads per process:
## should see 10 LSF jobs
# cluster = LSFCluster(queue='all_corradin',
#                          cores= 4,
#                          walltime='5000:00',
#                          memory='20GB',
#                          #local_directory = "/mnt/ramdisk",
#                          job_extra=['-o /dev/null', '-e /dev/null']
#                          )
# cluster.scale(100)
#----

print(f"SCALED TO {scale_num} workers")
print(cluster.job_script())

client=Client(cluster)

SCALED TO 564 workers
#!/usr/bin/env bash

#BSUB -J dask-worker
#BSUB -q all_corradin
#BSUB -n 47
#BSUB -R "span[hosts=1]"
#BSUB -M 240000000
#BSUB -W 9000:00
#BSUB -o /dev/null
#BSUB -e /dev/null

/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/.venv/bin/python -m distributed.cli.dask_worker tcp://172.18.102.9:8789 --nthreads 3 --nprocs 12 --memory-limit 18.63GiB --name dummy-name --nanny --death-timeout 60 --local-directory /tmp --protocol tcp://



In [6]:
# go to this link after 
print(f"go to this link after running the above to see the dashboard {cluster.dashboard_link}")

go to this link after running the above to see the dashboard http://172.18.102.9:8790/status


In [7]:
print(f"paste this link to the other script: '{client.scheduler.address}'")

paste this link to the other script: 'tcp://172.18.102.9:8789'


In [8]:
print(f"more info: {client.scheduler_info()}")

more info: {'type': 'Scheduler', 'id': 'Scheduler-a07532ed-158c-42c0-b33f-34a3334d5b46', 'address': 'tcp://172.18.102.9:8789', 'services': {'dashboard': 8790}, 'started': 1638546261.4680312, 'workers': {}}


In [5]:
cluster

/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/.venv/lib/python3.8/site-packages/distributed/scheduler.py:5490: UserWarning: Scheduler already contains a plugin with name worker-status; overwriting.
  warnings.warn(
tornado.application - ERROR - Exception in callback <function Cluster._widget.<locals>.update at 0x148d31432e50>
Traceback (most recent call last):
  File "/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/.venv/lib/python3.8/site-packages/tornado/ioloop.py", line 905, in _run
    return self.callback()
  File "/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/.venv/lib/python3.8/site-packages/distributed/deploy/cluster.py", line 382, in update
    maximum = IntText(0, description="Maximum", layout=layout)
  File "/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/.venv/lib/python3.8/site-packages/distributed/deploy/cluster.py", line 397, in _repr_html_
    adapt.on_click(adapt_cb)
  File "/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/.venv/lib/python3.8/sit

In [14]:
!bjobs -w -q all_corradin | grep PEND| wc -l

ValueError: filedescriptor out of range in select()

/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/.venv/lib/python3.8/site-packages/distributed/scheduler.py:5490: UserWarning: Scheduler already contains a plugin with name worker-status; overwriting.
  warnings.warn(


In [ ]:
!bjobs -w -q corradin | grep PEND| wc -l

In [ ]:
jobs = !bjobs -w -q all_corradin | grep PEND| cut -d " " -f1
jobs = jobs[int(len(jobs)/2):]
for job in jobs:
    !bswitch corradin {job}